In [1]:
si_id = [1,6,4,2,5,3,7,8,31,32,33,34,35,36,37,38,39]

In [5]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '쇼핑몰2'
# 파일 저장
f_dir = 'c:\\공항인근\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EC%87%BC%ED%95%91&choiceTagId=0f29b431-75ac-4ab4-a892-b247d516b31d')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
no = 1
cnttt = 50
page_cnt = math.ceil(cnttt/10)

for j in range(0,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()
    cnt = 1
    time.sleep(3)

    r_cnt= soup.find('div', class_='total_check').find('span').get_text( )
    r_cnt2 = r_cnt.replace(",","")
    search_cnt = int(r_cnt2)

    if cnttt > search_cnt :
        cnttt = search_cnt

    print("전체 검색 결과 건수 :",search_cnt,"건")
    print("실제 최종 출력 건수",cnttt)
    for i in range(1,page_cnt+1):
        
        for k in range(1,11):
            f = open(ff_name, 'a', encoding='UTF-8')
            
            if no > cnttt :
                break
                
            #각 게시글의 제목 누르기    
            title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
            driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
            time.sleep(random.randrange(5,7))

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')        
            content_list = soup.find('div','wrap_contView')    

            try:
                area_CD_tmp = soup.find('div', class_='area_address')
                area_CD = area_CD_tmp.find('span').get_text()
            except:
                area_CD_tmp = soup.find('div', {"id" : "topAddr"})
                area_CD = area_CD_tmp.find('span').get_text()
                
            try:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
                addr = addr.split('주소 ')
                    
                if (len(addr) == 1):
                    addr = addr[0]
                else :
                    addr = addr[1]
                    
            except IndexError:
                if (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                    
                elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
            
                elif(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text

                addr = addr.split('주소 ')
                    
                if (len(addr) == 1):
                    addr = addr[0]
                else :
                    addr = addr[1]

            print("no", no, ": ", title)
            print(area_CD)
            print(addr)
            f.write(str(no) + '//'+ str(title) + '//'+str(area_CD) +'//'+ str(addr) + "\n")
            f.close( )

            print("\n")
            no += 1
            no2.append(no)
            contents.append(title)
            area_CD2.append(area_CD)
            addr2.append(addr)

            driver.back( )  # 뒤로 돌아가기 기능
            time.sleep(random.randrange(5,7))
            cnt += 1
        if i > page_cnt+1:
            break
        i += 1
        
        if (i % 5 == 0):
            driver.find_element_by_link_text('''다음''').click()
        else :
            driver.find_element_by_link_text("""%s""" %i).click() # 다음 페이지번호 클릭    
        time.sleep(2)
        
tour = pd.DataFrame()
tour['no']=no2
tour['title']=contents
tour['area_cd']=area_CD2
tour['addr']=addr2

# csv 형태로 저장하기
tour.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
tour.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

전체 검색 결과 건수 : 996 건
실제 최종 출력 건수 50
no 1 :  （株）清云SP[韩国旅游品质认证/Korea Quality]（주식회사 청운에스피(CHEONGUN S.P Co.,Ltd.)[한국관광 품질인증/Korea Quality])
서울 서대문구
首尔特别市西大门区沙川路235


no 2 :  광장시장
서울 종로구
서울특별시 종로구 창경궁로 88


no 3 :  롯데면세점 (본점)
서울 중구
서울특별시 중구 을지로 30


no 4 :  롯데월드 쇼핑몰
서울 송파구
이용시간 10:00 ~ 20:00


no 5 :  KT&G 상상마당 디자인스퀘어
서울 마포구
서울특별시 마포구 어울마당로 65


no 6 :  핫트랙스 (광화문점)
서울 종로구
서울특별시 종로구 종로 1


no 7 :  핫트랙스 (잠실점)
서울 송파구
서울특별시 송파구 올림픽로 269


no 8 :  양재 하이브랜드
서울 서초구
서울특별시 서초구 매헌로 16


no 9 :  해밀톤 쇼핑센터
서울 용산구
이용시간 09:00 ~ 22:00


no 10 :  고속터미널 의류도매상가
서울 서초구
이용시간 08:00 ~ 21:30 (매장별로 다름)


no 11 :  동화면세점
서울 종로구
서울특별시 종로구 세종대로 149


no 12 :  퇴계로 오토바이상가
서울 중구
휴일 매장마다 다름


no 13 :  삼익패션타운
서울 중구
서울특별시 중구 남대문시장8길 7


no 14 :  꼼데가르송 (Comme des Garcons)
서울 용산구
이용시간 11:00 ~ 20:00


no 15 :  남대문 종합상가
서울 중구
서울특별시 중구 남대문시장4길 21


no 16 :  지오 송지오(ZIO SONGZIO) 문정점
서울 송파구
서울특별시 송파구 동남로4길 22


no 17 :  록시땅 (Loccitane)
서울 강남구
서울특별시 강남구 압구정로 162


no 18 :  청자 수입상가
서울 중구
휴일 연중무휴


no 19 :  퀸프라자
서울 중구
이용시간 점포별 상이


no 2

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]"}
  (Session info: chrome=85.0.4183.102)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.18363 x86_64)


In [3]:
#xPath로 게시글 수 가져온 것,, 먹힐까?

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt
from selenium.common.exceptions import NoSuchElementException
import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '쇼핑몰'
# 파일 저장
f_dir = 'c:\\캡디_공항인근\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EC%87%BC%ED%95%91&choiceTagId=0f29b431-75ac-4ab4-a892-b247d516b31d')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
view2 = []      # 게시글 내용 컬럼


no = 1

for j in range(0,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()

    time.sleep(3)

    r_cnt= driver.find_element_by_xpath('''//*[@id="totalCnt"]''').text
    r_cnt2 = r_cnt.replace(",","")
    search_cnt = int(r_cnt2)
    
    cnttt = 40
    page_cnt = math.ceil(cnttt/10)

    if cnttt > search_cnt :
        cnttt = search_cnt

    print("전체 검색 결과 건수 :",search_cnt,"건")
    print("실제 최종 출력 건수",cnttt)
    
    for i in range(1,page_cnt+1):
        
        for k in range(1,11):
            f = open(ff_name, 'a', encoding='UTF-8')
#             if no > cnttt :
#                 break
                
            #각 게시글의 제목 누르기    
            title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
            driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
            time.sleep(random.randrange(5,7))

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')        
            content_list = soup.find('div','wrap_contView')    
            con_1 = content_list.find('p').get_text()
            try:
                area_CD_tmp = soup.find('div', class_='area_address')
                area_CD = area_CD_tmp.find('span').get_text()
            except AttributeError:
                area_CD = '0'
            except:
                area_CD_tmp = soup.find('div', {"id" : "topAddr"})
                area_CD = area_CD_tmp.find('span').get_text()
            
            try:
                if(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text.split('주소 '))
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
                elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
            except IndexError:
                addr = '0'
            except NoSuchElementException:
                addr = '0'
            except:
                addr = '0'
            
            
            print("no", no, ": ", title)
            print(area_CD)
            print(addr)
            print(con_1)
            f.write(str(no) + ';'+ str(title) + ';'+str(area_CD) +';'+ str(addr) +';'+ str(con_1) + "\n")
            f.close( )

            print("\n")
            no += 1
            no2.append(no)
            contents.append(title)
            area_CD2.append(area_CD)
            addr2.append(addr)
            view2.append(con_1)
            
            driver.back( )  # 뒤로 돌아가기 기능
            time.sleep(random.randrange(5,7))
            
            if no > cnttt:
                break
        if i > page_cnt+1:
            break
        i += 1
        
        if no > cnttt:
            break
        
        if (i % 5 == 0):
            driver.find_element_by_link_text('''다음''').click()
        else :
            driver.find_element_by_link_text("""%s""" %i).click() # 다음 페이지번호 클릭    
        time.sleep(2)
    i=0
    k=0
    no=1
        
a = pd.DataFrame()
a['no']=no2
a['title']=contents
a['area_cd']=area_CD2
a['addr']=addr2
a['content']=view2

# csv 형태로 저장하기
a.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
a.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

전체 검색 결과 건수 : 258 건
실제 최종 출력 건수 40
no 1 :  주식회사 청운에스피(CHEONGUN S.P Co.,Ltd.) [한국관광 품질인증/Korea Quality]
서울 서대문구
서울특별시 서대문구 모래내로 235
서울 서대문구 남가좌동에 위치한 청운에스피는 홍삼 전문 판매장이다. 껍질을 벗기지 않은 수삼을 증기로 쪄서 말린 홍삼은 일반 인삼보다 사포닌이 풍부해 면역력 증진, 당뇨 예방 등에 탁월하다. 한국 홍삼은 외국인 관광객 사이에서 선물용으로 인기가 많다. 이곳에서는 국내 기준에 부합하는 특허기술 재료를 사용해 최상급 품질의 홍삼을 제조한다. 식품의약품안전처와 GMP(의약품 제조 및 품질 관리 기준) 인증 제품으로 안전하게 복용할 수 있다. 매장에는 홍삼을 비롯해 수삼을 물에 살짝 익혀서 말린 태극삼 등을 고급스럽게 포장해 판매한다. 바쁜 현대인이 편리하게 섭취할 수 있는 홍삼 캡슐, 분말, 엑기스, 정과 등 다양한 가공품도 진열되어 있다. 매장은 413㎡(약 125평) 규모로 최대 400명까지 수용 가능하며 연중무휴로 운영된다. 중국어, 영어, 말레이어, 광둥어 등 외국어 응대가 가능한 직원이 상시 근무 중이다. 매장에서는 수도권 전철 경의중앙선 가좌역이 가장 가까우며 한 정거장 거리에 홍대입구, 신촌, 영화 <어벤져스> 촬영지인 디지털미디어시티가 있다. 자동차로 10~15분 거리에는 하늘공원, 월드컵공원, 선유도공원이 위치해 관광을 즐기기에도 편리하다.


no 2 :  한국인삼공사 정관장 명동본점 [한국관광 품질인증/Korea Quality]
서울 중구
서울특별시 중구 남대문로 62
정관장 명동본점은 한국인삼공사에서 직영으로 운영하는 매장이며, 내외국인을 대상으로 하는 사후 및 사전 면세점이다. 중국, 홍콩, 대만 등에서 방문하는 고객의 비율이 높은 매장으로 중국어와 영어가 가능한 직원이 상시 근무한다. 외국어 가능 직원들은 정관장 제품에 대한 설명은 물론 본사에서 진행하는 일련의 프로모션과 이벤트 정보도 상세히 전한다. 일반 신

no 25 :  삼성모바일스토어 코엑스점
서울 강남구
서울특별시 강남구 영동대로 513
삼성모바일스토어 코엑스점은 삼성전자의 최신 IT 모바일 제품을 체험 및 구매할 수 있는 복합 서비스 매장이다.


no 26 :  영풍문고 종로점
서울 종로구
서울특별시 종로구 청계천로 41
영풍문고는 다양한 도서제공과 함께 낭독회, 사인회, 강좌 등 다양한 문화활동을 제공하는 문화공간이다. 영풍문고 종로점은 국내 서점 중 최대 규모인 3,200평으로 지하1층과 지하2층 매장으로 구성되어 있다. 지하1층에는 국내외 35만 여종의 도서를 구비하고 있으며, 도서정보 및 이용안내 그리고 회원가입, 택배발송 등을 할 수 있는 고객서비스센터가 마련되어 있다. 지하 2층에는 다양한 볼거리가 있는 ‘Stylish gift mall‘과 라이프스타일 브랜드 샵이 자리하고 있다. 디지털IT ZONE에는 ''삼성모바일 샵'' 국내1호점, 애플샵, 마이크로소프트 브랜드샵이 입점해 있다. 또한 음반, 뷰티∙헬스케어용품, 고급만년필, 다이어리, 디자인문구, 디자인소품, 팬시용품, 캐릭터 용품, 리빙용품 등 다양한 상품이 구비되어 있다.


no 27 :  센트럴시티
서울 서초구
서울특별시 서초구 신반포로 176
‘도심 속 또 하나의 도시’로서의 기능을 하는 고품격 복합생활문화공간인 센트럴시티는 서울시 강남구 반포동에 위치해있다. 신세계 백화점, 메리어트 호텔을 비롯하여 휴식공간인 센트럴파크가 자리하고 있고, 예식장과 대형 행사를 치를 수 있는 밀레니엄 홀과 은행, 병원, 증권, 우체국 등의 편의시설 및 대형주차 공간도 갖추어져 있어 복합문화시설로 한국의 건축 문화를 한차원 높이는 수준의 복합 건물로 평가되고 있다. 또한 센트럴시티 안에는 고품격 명품 쇼핑몰 마르퀴스 플라자, 다양한 장르의 음반을 취급하는 신나라 레코드, 초대형 문고센터 영풍문고, 세계적으로 대 유행인 선진국형 멀티플렉스 센트럴 6 시네마와 시뮬레이션을 비롯한 첨단 오락시설을 갖춘 도심형 테마파크 등이 자리 잡고 있어 신세대들이 많이

no 14 :  부산 자갈치시장
부산 중구
부산광역시 중구 자갈치해안로 52
자갈치시장은 중구 남포동과 충무동에 걸쳐 있는 해안통으로 도심과 접하고 있어 시장기능과 위락기능을 겸하고 있을 뿐만 아니라 인근의 신동아 수산물시장, 건어물시장과 함께 부산지역의 성격을 가장 잘 보여주는 시장이다. 자갈치라는 지명은 자갈해안에서 비롯 되었다는 설과 활어만을 취급한다고 하여 활어로서 많이 거래되는 "자갈치"란 어종의 명칭에서 유래되었다는 설이 있다.자갈치 해안가 도로변에는 "판매기장수 아주머니"들이 고등어, 멍게, 고래고기 등을 파는 진귀한 모습을 볼 수 있고 1980년에 문을 연 "신동아수산물시장" 1층에는 활어 판매와 즉석에서 싱싱하고 맛있는 회를 맛볼 수 있으며 2, 3층에는 건어물 및 해산물 판매와 생선횟집 이 자리하고 있어 새로운 자갈치 시장의 명물로 떠오르고 있다. 이 곳의 장이 성장하게 된 것은 1889년 일본인들이 자국어민을 보호하기 위해 인근에 부산수산주식회사를 세우면서부터였다. 그곳으로 상인들이 몰리자 자갈치시장 상인들도 서서히 근대화를 모색하게 되었는데, 1922년에 이르러 부산 어업협동조합이 남포동에 건물을 짓고 위탁판매 사업을 시작함으로써 자갈치시장의 상인들이 구심점을 찾아 모여 들었다.이에 따라 부산의 어시장은 북항의 부산수산주식회사와 남항의 부산어협 위탁판매장으로 양분되었는데, 그 뒤 부산수산주식회사는 국내 최대의 어시장인 현재의 부산 공동어시장으로 발전하게 되었고, 남항에 출어하는 영세어선들의 어획물을 다루는 영세상인들이 부산어협 위탁판매장 주변에 모여 지금의 자갈치시장을 이루었다.


no 15 :  국제시장
부산 중구
부산광역시 중구 신창로4가 일원
부산 지하철 1호선인 자갈치역 7번출구와 남포동역 1번출구에 인접한 곳이다. 1945년 광복이 되자 일본인들이 철수하면서 전시 물자를 팔아 돈을 챙기기 위해 국제시장 자리를 장터로 삼으면서 시장이 형성되었다. 한국전쟁 후 피난민들이 장사를 하며 활기를 띠었고, 미군의 군용 물자와 함께 부산항으

no 38 :  은송타미나
부산 중구
0
25년 넘는 전통을 자랑하는 광복동 시장 내의 전통가구 및 공예품 판매점으로, 고가구와 도예작품을 비롯하여 여러 가지 인테리어 소품을 판매하고 있다. 가격이 저렴하며, 다양한 인테리어 소품을 한 장소에서 구매가 가능하여 일본 관광객이 특히 많이 찾으며, 전화주문을 통한 택배 서비스가 가능하다.


no 39 :  남포동건어물도매시장
부산 중구
0
부산 바다의 냄새가 눅진하게 배어있는 전통 재래시장이다. 위탁 장사를 하던 이전의 모습에서 세월이 흐름에 따라 지금은 소매 상인들이 대부분이지만, 비릿하면서 구수한 부산의 냄새를 맡고 싶다면 방문해 볼 필요가 있는 곳. 자갈치시장 동쪽 끝에 위치한 건어물 도매시장으로 건어물을 전문적으로 취급하는 시장이다. 부산시 유일의 수협공판장이 있으며, 이곳의 200여 개 점포에서 마른 멸치, 마른 오징어, 새우, 각종 포 등 건어물과 김, 미역, 조개 등 해산물 등을 판매하고 있다. 전국의 중간 도매상들은 물론 일반 소비자들에게도 싼값으로 도소매하는 부산 제일의 건어물 시장이다.


no 40 :  이상봉 (롯데백화점 센텀시티점)
부산 해운대구
부산광역시 해운대구 센텀남대로 59
이상봉의 컬렉션은 여성적 실루엣을 구조적, 기하학적으로 완벽하게 구현하여 여성적 아름다움을 최대한으로 표현하되 일상에서 자연스럽게 착용이 가능한 창작으로 구현하고 있다. 특히 빨강과 보라의 과감한 컬러감은 디자이너가 사용하는 비비드한 컬러군과 조화를 이루며 강렬한 에너지를 발산한다.


전체 검색 결과 건수 : 19 건
실제 최종 출력 건수 19
no 1 :  현풍장 (5, 10일)
대구 달성군
0
현풍장은 1918년에 개설되어 90년의 역사를 지니고 있다. 규모면에서 대구.경북을 대표하는 5일장 정기시장중 하나였으며, 2011년 9월부터 토.일 주말시장을 운영하고 있다. 고령. 창녕. 청도 등의 인근지역에서 많이 이용하고 있다. 현풍장이 열리는 시장위치는 현풍면사무소와 약 600m 거리에 있으며, 시장옆에 달성

전체 검색 결과 건수 : 31 건
실제 최종 출력 건수 31
no 1 :  올리브 아울렛 (인천점)
인천 남동구
0
올리브 (Allive : All + live) - 생활 속의 모든 것을 쇼핑 할 수 있는 아울렛 올리브아울렛의 새로운 기업마크인 절제되고 감각적인 디자인의 영문 ''Allive''는 매 우 유연하고 세련된 All+live의 합성어로써 인간생활의 편익과 모든 것을 쇼핑할 수 있다는 의미를 표현하고자 하였으며, 고객과의 교감(Communication) 을 통하여 항상 고객과 함께하는 서비스 기업으로서의 올리브를 의미하고 있다.


no 2 :  인천 모래내시장
인천 남동구
인천광역시 남동구 호구포로810번길 42-8
모래내시장은 인천에 있는 시장 중 큰 시장으로 꼽히며 유동인구 또한 많은 시장이다. 구월동 농산물 도매시장과 가까운 거리 탓인지 싱싱한 농산물을 싼 가격에 구입할 수 있어 주변 알뜰 주부들은 물론, 재래시장의 향취와 인심에 매료된 관광성 고객들의 발길도 잦다. 남동구에서 최대 규모를 자랑하는 재래시장이다. 또한 상인들의 후한 인심을 몸소 느껴본다면 다시 안 올 수 없는 곳이며, 방송에도 여러번 촬영을 했던 곳이다. 연안부두와 소래포구에서 긴급 공수한 수산물도 빼놓으면 섭섭할 자랑거리이다.


no 3 :  엔타스면세점
인천 중구
인천광역시 중구 영종해안남로321번길 186
인천을 대표하는 엔타스면세점은 인천시내면세점, 인천공항점, 인천항만점을 운영하고있다. 운서동에 위치한 인천 본점은 인천 유일의 시내면세점으로 패션 명품브랜드를 비롯해 시계, 주얼리, 화장품, 향수, 선글라스, 그리고 국산품까지 다양한 상품을 판매하고 있다.


no 4 :  부평모두몰(부평지하도상가)
인천 부평구
0
부평역 지하에 위치한 부평모두몰(부평지하도상가)은 국내에서 손꼽히는 지하 쇼핑 센타이다. 미국 월드레코드아카데미로 부터 ''단일 면적 최다 지하상가 점포 수''로 세계기록 인증을 받았다. 부평지하상가는 A동부터 G동까지 나눠져 있으며, 점포 수만 1,400

...

no 4 :  남원 목운공예사
전북 남원시
전라북도 남원시 요천로 1230-9
흔히 일본을 칠기의 나라라고 한다. 소품에 들어가는 옻칠한 찻잔 세트가 100만 원대를 훌쩍 넘기는 경우는  흔하고 기능을 갖춘 옻칠장도 사회적으로나 경제적으로 굉장한 대우를 받는다. 그런  일본도 절대 따라오지 못하는 부분이 우리 전통 옻칠에 있다. 바로 칠을 정제하는  분야다. 나무에서 채취한 생칠은 수분과 각종 불순물을 함께 담고 있다. 우리 전통  옻칠은 이 생칠을 거르고 걸러 순도 95%까지 정제해낸다. 한 해 20억 원어치 정도 수입되는 일본산이 고작 35% 내외의 순도인 것에 비춰보면 엄청난 차이다. 그야말로  옻칠 정제에 관한 한 세계 최고다. 옻칠은 고순도일수록 투명하고, 값도 비싸다. 남원 목운공예사 박강용 운영자는 바로 그런 옻칠을 정제할 줄 아는 몇 안 되는 옻칠장이다. 전북도 지정 제13호 무형문화재이다. 옻칠 중에서도 정제 분야의 전문가다. 박 운영자는 국가 지정 중요문화재인 제113호 정수화 옻칠장의 기능 전수자다. 어려서 옻칠 세계에 뛰어들어 15년이나 기본을 익힌 뒤 정수화 스승에게 따로 7년을 전수받았다고 한다. 박 운영자는 음식물을  담는 칠기에는 반드시 이런 고순도 옻칠이 필요하다고 한다. 고순도 옻칠을 하면 열전도율이 매우 낮아 섭씨 200도의 고온에도 견디기 때문에 색의 변화도 없다. 일반 칠은 섭씨 60도를 견디지 못한다. 박 운영자는 일단 역한 냄새가 나는 옻칠 목기는 합성칠을 쓴 것으로 보면 된다고 알려준다. 정제 과정에서 포르말린 성분이 든 희석제를 써야 하기 때문이다. 값싼 중국산 목기에서 독한 냄새가 나면 틀림없다. 일본은 식기류에 합성칠을 사용하는 것을 전면 금지하고 있다. 우리 전통기법을 제대로 살린 옻칠기는 그 효능이 무궁무진하다. 피톤치드 향이 함유돼 가까이 두면 가벼운 두통 정도는 금세 사라진다. 방습, 항균, 방충, 방독, 방염에 전자파 차단 효과까지 낸다. 박 운영자는 전통 옻칠기의 디자인과 문양을 현대적으로 재해석하는 

no 29 :  장수장 (5, 10일)
전북 장수군
0
대규모의 특산물 재배단지는 없고, 소규모로 생산.판매되며, 주요 특산물은 계절에 따 라 다르게 상거래된다. 이 지역 장터에서 특히 눈에 띄는 것은 석기이다. 장수 곱돌그 릇은 밥을 짓거나 음식을 끓이면 맛도 좋을 뿐만 아니라 잘 식지도 않기 때문에, 전국 적으로 유명하고 인기가 높다.  곱돌그릇은 일년 내내 구입이 가능하며, 상품은 장수 장터에 건너편에 판매장이 몇 군 데 있다. 대성리 구암부락에 가면 ''장수석기''라는 곱돌그릇 공장이 있는데, 그릇 만드 는 과정도 구경하고 싼값에 구입할 수 있어 가볼 만하다. 또한 장수에는 ''사과''가 많이 산출된다. 밤낮의 기온차가 큰 탓에 장수에서 나는 사과 는 달고 야물다.


no 30 :  정읍 신태인시장
전북 정읍시
전라북도 정읍시 신태인읍 시장2길 14
신태인은 1940년 대전광역시와 같이 읍(邑)으로 승격한 도시로 일제강점기에는 호남지방에서 곡물 거래량이 가장 많은 도시로 손 꼽혔다. 신태인 시장은 1910년대 부터 5일장으로 열리고 있으며, 2008년 12월 30일 시장 현대화 사업을 마무리하고, 새롭게 개장되었다. 신태인 시장의 특징은 수산물 가격이 대체적으로 저렴하게 형성되고 있으며, 고추 주산지로 좋은 고추와 신태인 농협 청결 고춧가로 가공공장에서 생산되는 양질의 고춧가루를 구입할 수 있다는 특징이 있다.


no 31 :  손내옹기
전북 진안군
0
손내옹기는 진안군 백운면 평장리 정송마을에 위치해 있으며 전주한옥마을과 롯데 백화점 본점, 인사동 쌈지길 공예 골목에 매장이 있다. 손내옹기의 손내는 옹기 빚는 장이의 손길 또는 옹기마을인 솥내의 이중적 의미를 담고 있다. 손내옹기는 진안고원형 옹기장의 기술 보유자로 인정받은 이현배 옹기장이 운영 하고 있다. 손내옹기의 달항아리와 전골솥은 2008년에 유네스코에서 우수수공예품으로 인증 받았다.


no 32 :  정일품공방
전북 남원시
전라북도 남원시 요천로 1268
정일품공예관은 우리 조상의 혼이 

no 11 :  나주목사고을시장
전남 나주시
전라남도 나주시 청동길 14
나주목사고을시장은 과거 호남의 도읍지가 있던 나주에 개설되어 있는 전통시장이다. 상설시장인금계매일과 정기시장인 성북시장을 통합해 2012년에 새롭게 개설했다. 계절마다 달라지는 나주의 특산물과 농산물, 생활필수품 등을 판매하고 있다.


no 12 :  보성장 / 녹차골 보성향토시장 (2, 7일)
전남 보성군
0
보성시장이 개장되는 매 2일과 7일은 전국 시군청 소재지 시장이 갖는 공통점이다.1941년 행정구역이 개편되기 이전에 용문면이라 하여, 지금의 보성읍 용문리 우막에 조선조 말엽부터 우시장이 개설되어 정기시장으로 발전하였으며 1930년대에 경전서부선 철도가 개통되자 보다 교통이 편리한 보성읍 보성리 지금의 보성농협 자리에 이설하였다가 4~5년후 지금의 보성읍사무소 부근으로 옮겼었다. 그 후 1950년대 초반 보성읍 원봉리 현 시장으로 옮겨 오늘에 이르고 있는데,74년 시장 새마을 운동의 일환으로 장옥 전체를 개수하고 상가를 신축하여 시장으로 자리하였는바 지금도 전국에서 제일 큰 마포베 시장이 새벽 일찍 서고 있으며, 하룻장에 최고 2천여필이 거래 되는 것 으로 알려지고 있다.보성 우시장은 남원의 성우시장과 함께 송아지 시장으로 유명하였고, 또한 장날이면 소장수를 따라 멀리 진주,남원의 우시장까지 도보로 소를 몰아다 주는 전문 소몰이꾼으로 우시장이 북적대었으며, 몰이꾼은 한사람이 평균 4~5마리를 모는게 보통인데 실력있는 몰이꾼은 열마리까지도 소를 몰았었다.거래두수에 있어서는 70년대까지는 1일 평균이 100두를 상회하였지만 지금은 전업양축농가의 증가와 교통의 발달로 인하여  거래양상이 달라져 평균 40여두 정도로 거래 규모가 위축되고 있다. 그러나 정기시장내 편의시설등은 아직도 원시성을 벗어나지 못하고 있고 노점상이 활개하고 있지만 정기시장에서 거래되는 물량은 상당할 뿐 아니라 정기시장일이면 주변 상가들이 활기를 띠고 있다.


no 13 :  여수 종합수산시장
전남 여수시
0
여

no 36 :  벌교장 (4, 9일)
전남 보성군
0
4일과 9일에 장이 서는 벌교시장은 조선조 말엽 남하면의 좌촌시장이 지금의 벌교읍 장좌리에 매 9일에 개장하는 10일장으로 섰고, 낙안현의 벌교기장이 역시 10일장으로 매 4일에 지금의 벌교읍 고읍리에 장이 섰는데, 곡물시장이 주류를 이루고 생활품 등 잡화가 거래되었다. 좌촌시장과 벌교시장을 합하여 4일과 9일에 개장하게 된 것은 정확한 자료가  없어 알 수가 없고, 1920년 무렵 별교수산주식회사가 일인들에 의해 설립되고 이를 중심으로  어시장이 매일 개시되는 등 발전하기 시작하여, 1930년대에 이르러서는 인근 고흥반도에서 생산되는 농.수산물을 일본으로 수송할 목적으로 철도의 개통과 더불어 벌교가 거점상업 도시로 성장하게 됨에 따라 정기시장도 자연히 활기를 띄게 되었다. 광복 후 1947년 지금의 벌교읍 벌교리 2번지 일대의 세망시장으로 이설하였고 이때에는 승주. 고흥군의 일부와 여천군 일부 주민들까지도 이용함으로써, 시장과 함께 전남 동부지역의 중심시장으로 자리 잡았었다. 1956년 벌교읍 회정리 663번지 일대의 지금의 시장으로 옮기고 어시장이 매일 시장화 되었는데 꼬막 등 어패류가 년간 800여 톤이 거래되고 있으며, 이는 대부분 전문 도매상들에 의해 서울, 광주 등 대도시로 직송되고 있다. 우시장도 1970년대에만 하더라 1일 평균 거래가 200두를 상호하여 1980년대초 벌교읍 봉림리에 별도의 우시장을 개설하였으나, 지금은 여건의 변화로 30~40두가 거래 되는 등 점차 활기를 잃어가고 있다. 벌교시장은 1993년 벌교시장 현대화 사업으로 특별교부세 3억원을 지원받아 상가건물 신축과 화장실등 편의시설을 설치함으로써 현대시장으로 탈바꿈 하고 있다.


no 37 :  함평장 (2, 7일)
전남 함평군
전라남도 함평군 함평읍 시장길 49-1
전남 함평장은 예로부터 유명한 장터이다. 함평장을 가리켜 ''함평 큰 소장''이라고도 부르는데, 이것은 함평의 우시장이 크기 때문에 붙여진 별명이다. 그만큼 함

no 22 :  ㈜다원코리아 [한국관광 품질인증/Korea Quality]
제주 제주시
제주특별자치도 제주시 도리로 40
다원코리아는 외국인 전용 면세점을 운영하는 기업이다. 지상 2층 규모에 총 8개 쇼룸으로 구성된 외국인 전용 면세점과 중국, 홍콩, 말레이시아, 마카오에 현지 총판을 운영한다. 다원코리아는 업계 최초로 간 기능 개선과 관련해 특허 등록된 헛개나무 열매를 원료로 사용한 건강기능식품을 선보였다. 또 국내 최초로 식품의약품안전처로부터 간 관련 기능성 원료 개별 인정을 획득해 알코올성 손상으로부터 간을 보호하는 기능을 인정받았다. 특허기술 확보를 통한 차별화된 상품 개발이야말로 다원코리아가 만든 상품의 신뢰도를 높이는 바탕이 된다. 다원코리아는 제품 판매 향상을 위해서도 노력하고 있다. 체계적인 고객만족(CS) 교육을 통해 고객 눈높이 서비스를 시행하며, 판매 후 관리 역시 체계적으로 이뤄진다. 최근 단체 관광객보다 개별 자유여행자가 늘어나는 여행업 변화에 맞춰 개별 여행자를 위한 다양한 상품 구비, 서비스 전략을 펼치고 있다.  


no 23 :  ㈜제주참앤참 [한국관광 품질인증/Korea Quality]
제주 제주시
제주특별자치도 제주시 한천로 29
(주)제주참앤참은 화장품 전문 면세점을 운영한다. 동남아시아 개별 자유여행자와 단체 관광객을 대상으로 하며, 주력 상품은 홍삼을 주재료로 만든 코즈메틱 브랜드 구본초의 제품들이다. 이 외에 한국에서 유통, 판매되는 화장품을  판매한다. 구본초 화장품은 인삼과 홍삼, 산삼 등 한방 재료를 이용해 만든 고품격 한방 화장품이다. 콜라겐 마스크 팩과 기능성 제품인 앰풀류, 각종 크림 등 다양한 제품군을 갖췄다. 이 상품들을 제주참앤참에서 만날 수 있다. 이 밖에 코리아나, 미샤, 라네즈, 더페이스샵 등 국내외에서 인기가 높은 다양한 화장품 브랜드도 함께 다룬다. 1층은 단독 매장 형태로 꾸며졌으며 넓고 쾌적해 쇼핑하기 좋다. 점원 모두 중국어를 구사해 중화권 관광객들이 이용하기 편하다.  


no 24

In [7]:
#xPath로 게시글 수 가져온 것,, 먹힐까?

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt
from selenium.common.exceptions import NoSuchElementException
import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '쇼핑몰'
# 파일 저장
f_dir = 'c:\\캡디_공항인근\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EC%87%BC%ED%95%91&choiceTagId=0f29b431-75ac-4ab4-a892-b247d516b31d')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
view2 = []      # 게시글 내용 컬럼


no = 1

for j in range(2,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()

    time.sleep(3)

    r_cnt= driver.find_element_by_xpath('''//*[@id="totalCnt"]''').text
    r_cnt2 = r_cnt.replace(",","")
    search_cnt = int(r_cnt2)
    
    cnttt = 40
    page_cnt = math.ceil(cnttt/10)

    if cnttt > search_cnt :
        cnttt = search_cnt

    print("전체 검색 결과 건수 :",search_cnt,"건")
    print("실제 최종 출력 건수",cnttt)
    
    for i in range(1,page_cnt+1):
        
        for k in range(1,11):
            f = open(ff_name, 'a', encoding='UTF-8')
#             if no > cnttt :
#                 break
                
            #각 게시글의 제목 누르기    
            title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
            driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
            time.sleep(random.randrange(5,7))

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')        
            content_list = soup.find('div','wrap_contView')    
            con_1 = content_list.find('p').get_text()
            try:
                area_CD_tmp = soup.find('div', class_='area_address')
                area_CD = area_CD_tmp.find('span').get_text()
            except AttributeError:
                area_CD = '0'
            except:
                area_CD_tmp = soup.find('div', {"id" : "topAddr"})
                area_CD = area_CD_tmp.find('span').get_text()
            
            try:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            except IndexError:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            except NoSuchElementException:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            except:
                addr = '0'
            
            
            print("no", no, ": ", title)
            print(area_CD)
            print(addr)
            print(con_1)
            f.write(str(no) + ';'+ str(title) + ';'+str(area_CD) +';'+ str(addr) +';'+ str(con_1) + "\n")
            f.close( )

            print("\n")
            no += 1
            no2.append(no)
            contents.append(title)
            area_CD2.append(area_CD)
            addr2.append(addr)
            view2.append(con_1)
            
            driver.back( )  # 뒤로 돌아가기 기능
            time.sleep(random.randrange(5,7))
            
            if no > cnttt:
                break
        if i > page_cnt+1:
            break
        i += 1
        
        if no > cnttt:
            break
        
        if (i % 5 == 0):
            driver.find_element_by_link_text('''다음''').click()
        else :
            driver.find_element_by_link_text("""%s""" %i).click() # 다음 페이지번호 클릭    
        time.sleep(2)
    i=0
    k=0
    no=1
        
a = pd.DataFrame()
a['no']=no2
a['title']=contents
a['area_cd']=area_CD2
a['addr']=addr2
a['content']=view2

# csv 형태로 저장하기
a.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
a.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

전체 검색 결과 건수 : 19 건
실제 최종 출력 건수 19
no 1 :  현풍장 (5, 10일)
대구 달성군
대구광역시 달성군 현풍면 비슬로130길 53-1
현풍장은 1918년에 개설되어 90년의 역사를 지니고 있다. 규모면에서 대구.경북을 대표하는 5일장 정기시장중 하나였으며, 2011년 9월부터 토.일 주말시장을 운영하고 있다. 고령. 창녕. 청도 등의 인근지역에서 많이 이용하고 있다. 현풍장이 열리는 시장위치는 현풍면사무소와 약 600m 거리에 있으며, 시장옆에 달성군 보건소가 위치하고 있고 현풍 IC에서 약 4km거리에 있다. 시장이 위치한 현풍지역은 비슬산(1,084m)과 낙동강을 끼고 있어 풍광이 아름답기로 유명하며, 마산~대구간의 국도와 진주~대구 간의 지방도가 면의 남서부에서 교차하고 있어 교통이 편리하다. 또한, 지역 관광지(달성보, 현풍석빙고, 현풍향교, 유가사, 소재사, 비슬산 자연휴양림, 도동서원, 곽망우당묘소, 십이정려각 등)와 연계하여 농.특산물을 쇼핑할 수 있는 전통적인 재래시장이다. 장터가 열리면 이웃하고 있는 유가면, 구지면, 논공읍은 물론 고령, 창녕, 청도 등의 각종 특산품과 농산물이 집결되는 등 큰장이서고 있다.


no 2 :  대림목공예
대구 동구
대구광역시 동구 공항로33길 21
대림목공예는 1983년 10월 설립을 하여, 지금까지  오직 목공예 제품만을 생산해왔다. 처음에는 서민적인 멋과 현대적 감각에 어울리는  제품들을 주로 생산하여 왔으며, 현재는 세월의 흐름과 시대의 변화에 따라 소비자가  요구하는 모든 분야에 만족을 위해 최첨단 레이저 조각기(미국 에필로그시 : RADIOS)를 도입하여 다방면에 활용을 하고 있으며, 목공예품을 우리 실생활에 접목하여 우리 고유의 문화유산을 지켜나간다.


no 3 :  불로장 / 불로전통시장 (5, 10일)
대구 동구
대구광역시 동구 팔공로26길 9-1
불로전통시장이 위치한 불로동은 한자로 不老洞, 늙지 않은 동네란 뜻이다. 왕건이 동수전투(일명:공산전투)에서 패하여 도주하다

KeyboardInterrupt: 